# Cloud Ingestion into Snowflake

## 🚧 SETUP ZONE 🚧

### Environment Options

**Option A:** If you are comfortable locally navigating the terminal, Jupyter notebooks, Python versions, using pip (or conda), and/or creating and managing virtual environments, follow the instructions in the cell marked "Option A: Setup Instructions" below. **This option is for intermediate to advanced Pythonistas.**

**Option B:** If you'd prefer **NOT** to deal with locally navigating the terminal, Jupyter notebooks, Python versions, pip (or conda), and/or virtual environments, simply log into your Snowflake account and easily import this notebook into your Snowflake environment. You can do this by following the instructions in the cell marked "Option B: Setup instructions" below. **This option is for beginner to intermediate to otherwise can't-be-bothered Pythonistas.**

### Option A: Setup Instructions

1. [Download the notebook here](https://tinyurl.com/pydata2025-nb) (OR clone the repo to your computer and navigate to it in the terminal):

```bash
git clone https://github.com/Snowflake-Labs/pydata_boston_2025_notebook_to_pipeline.git && cd pydata_boston_2025_notebook_to_pipeline
```

2. Create and activate a Python virtual environment to isolate dependencies:

```bash
# Create a virtual environment
python3 -m venv pydataboston_2025

# Activate the virtual environment
# On macOS/Linux:
source pydataboston_2025/bin/activate

# On Windows:
pydataboston_2025\Scripts\activate
```

3. Install all required packages using pip:

```bash
pip install --upgrade pip
pip install -r requirements.txt
```

**Alternative:** Install packages individually:

```bash
pip install snowflake-snowpark-python>=1.11.0
pip install snowflake-core>=0.6.0
pip install pandas>=1.5.0
```

4. Verify that the Snowflake packages are installed correctly:

```python
python3 -c "from snowflake.snowpark import Session; from snowflake.core import Root; print('Installation successful!')"
```

### Option B: Setup Instructions

1. Log into the Snowflake account you created.

2. In the left menu, hover over **Projects** and click on **Notebooks**.

3. At the top right, click on the dropdown on the **+ Notebook** button, and select "import .ipynb file".

4. Fill out the modal:

- Give the notebook a name
- Select **SNOWFLAKE_LEARNING_DB** as your database, and **PUBLIC** as the schema
- For runtime, select **Run on warehouse**
- Leave everything else as-is

5. Click **Create**. Standby for further instructions.

6. Using the **Packages** drop down at the top, search for `snowflake` and click on it. This will install the `snowflake` Python package in the notebook environment. Next, search for `snowflake-snowpark-python` and click on it. This will install Snowpark in your notebook environment.

7. Proceed to the cell 9.

## ✅ END OF SETUP ZONE ✅

Let's get started! I'll be using Option A, but I'll be able to guide those who are using either Option A or Option B.

First, we'll import the necessary libraries. We'll use:

- `snowflake.core` for data operations using Snowflake's Python APIs and Snowflake's compute engine (won't run into any local memory errors)

- `snowflake.snowpark` for Python DataFrame operations

In [25]:
from snowflake.snowpark import Session # For Snowflake compute engine connection
from snowflake.core import Root, CreateMode # For core operations
from snowflake.core.database import Database # For database operations
from snowflake.core.schema import Schema # For schema operations
from snowflake.core.warehouse import Warehouse # For compute engine operations
from snowflake.core.table import Table, TableColumn, TableCollection # For table operations
from snowflake.core.role import Role, Securable # For role operations
from snowflake.core.stage import Stage, StageDirectoryTable # For stage operations
from snowflake.core.dynamic_table import DynamicTable, DownstreamLag, UserDefinedLag, DynamicTableCollection # For dynamic table operations
from snowflake.snowpark.types import * # For defining DataFrame schema
from snowflake.snowpark import functions as F # For DataFrame functions

### Connect to Snowflake

We'lls use Snowflake's compute engine for this lab. This way we can take advantage of distributed computing and not be bounded by any limits of our computer's local memory when loading or processing (transforming) data. 

Start by creating a session to connect to Snowflake:

You can either:

1. Pass credentials directly in a connection parameters dictionary (not recommended for production – but we will do this just for today's lab) 
2. Use a connection configuration file using Snowflake CLI
3. Use environment variables or external authentication

For this tutorial, **we'll use a connection parameters dictionary**. Again, this is not recommended for production, but we will do this only for today's tutorial.

![Snowflake Login](./img/snowflake_login.png)

In [26]:
# THIS CELL FOR OPTION A ONLY

# Replace these with your Snowflake account details
# Follow along with Gilberto for guidance on how to fill these out

connection_parameters = {
    "account": "DJKRPAC-QHB69982",
    "user": "JOAOMARIO001",
    "password": "c4auehbKxWAD7nq*R",
    "role": "ACCOUNTADMIN",
    "warehouse": "COMPUTE_WH"
}

# Create a session
session = Session.builder.configs(connection_parameters).create()
print(f"❄️ Connected to Snowflake account: {session.get_current_account()}")
print(f"Current role: {session.get_current_role()}")

❄️ Connected to Snowflake account: "DJKRPAC-QHB69982"
Current role: "ACCOUNTADMIN"


In [29]:
# # THIS CELL FOR OPTION B ONLY

# from snowflake.snowpark.context import get_active_session
# session = get_active_session()

# print(f"❄️ Connected to Snowflake account: {session.get_current_account()}")
# print(f"Current role: {session.get_current_role()}")

### Create Root API Object

The Root object is the entry point for Snowflake's Python API. It provides access to all database objects and operations. We must instantiate `Root` to be able to do anything with the data in Snowflake.

In [30]:
# Create root object from Root():
root = Root(session)
print("Root API object created successfully")

Root API object created successfully


### Create Dedicated Lab Role Using Python API

We'll create a dedicated role for this lab with the necessary privileges using the Snowflake Python API. This follows the principle of least privilege, meaning that a user in a data platform should have only the minimum required privileges to execute specific jobs or tasks. **In practice, this is known as "Role Based Access Control", frequently referred to as "RBAC".**

In [31]:
# Create lab role using Python API
pydata_lab_role = Role(name="pydata_lab_role")
root.roles.create(pydata_lab_role, mode=CreateMode.if_not_exists)
print("Role 'pydata_lab_role' created successfully")

# Grant role to SYSADMIN (SQL required for role grants)
session.sql("GRANT ROLE pydata_lab_role TO ROLE SYSADMIN").collect()

# Grant necessary privileges to role (SQL required for account-level grants)
session.sql("GRANT CREATE WAREHOUSE ON ACCOUNT TO ROLE pydata_lab_role").collect()
session.sql("GRANT CREATE DATABASE ON ACCOUNT TO ROLE pydata_lab_role").collect()

print("Privileges granted to pydata_lab_role")

Role 'pydata_lab_role' created successfully
Privileges granted to pydata_lab_role


### Switch to/assume lab role

Now we'll assume our new **pydata_lab_role** for all subsequent operations.

In [32]:
session.use_role("pydata_lab_role")
print(f"Switched to role: {session.get_current_role()}")

Switched to role: "PYDATA_LAB_ROLE"


### Review data pipeline architecture

Let's quickly review the reference architecture diagram to understand how we'll build the pipeline.

## Part 1: Define data objects

### Create Database Using Python API

We'll create a database called `tasty_bytes_db`. This database will contain all of the Tasty Bytes sales data that we'll use to build our pipeline.

In [ ]:
# Create database
database_name = "tasty_bytes_db"
new_database = Database(name=database_name)
root.databases.create(new_database, mode=CreateMode.or_replace)

print(f"Database '{database_name}' created successfully")

# Set database context
session.use_database(database_name)
print(f"Session is now using database: {session.get_current_database()}")

Database 'tasty_bytes_db' created successfully
Session is now using database: "TASTY_BYTES_DB"


### Create Schemas

A schema is a grouping of data within a database. They're used to logically group related sets of data. For this lab, we'll create two schemas: 

* `raw` (for source data)
* `analytics` (for transformed data)

This separation of raw and transformed data is a common pattern in the field of data engineering. Schemas can be named anything you want.

![Schemas Creation](./img/schemas_creation.png)

In [ ]:
# Get database reference
db = root.databases[database_name]

# Create raw schema
raw_schema = Schema(name="raw")
db.schemas.create(raw_schema, mode=CreateMode.or_replace)
print("Schema 'raw' created successfully")

# Create analytics schema using Python API:
analytics_schema = Schema(name="analytics")
db.schemas.create(analytics_schema, mode=CreateMode.or_replace)
print("Schema 'analytics' created successfully")

Schema 'raw' created successfully
Schema 'analytics' created successfully


### Create Compute Resource for Processing Data

A virtual warehouse provides compute resources for query execution in Snowflake. We'll create an X-Large warehouse with auto-suspend to optimize costs.

![Virtual Warehouse](./img/virtual_wh.png)

In [ ]:
# Create compute resource
virtual_warehouse_name = "pydata_lab_wh"

new_warehouse = Warehouse(
    name=virtual_warehouse_name,
    warehouse_size="XLARGE"
)

root.warehouses.create(new_warehouse, mode=CreateMode.or_replace)
print(f"Warehouse '{virtual_warehouse_name}' created successfully")

# Grant usage on warehouse to pydata_lab_role
root.roles["pydata_lab_role"].grant_privileges(
    privileges=["USAGE"],
    securable_type="WAREHOUSE",
    securable=Securable(name=virtual_warehouse_name)
)
print(f"Granted USAGE on warehouse to pydata_lab_role")

# Use the warehouse
session.use_warehouse(virtual_warehouse_name)
print(f"Using virtual warehouse: {session.get_current_warehouse()}")

Warehouse 'pydata_lab_wh' created successfully
Granted USAGE on warehouse to pydata_lab_role
Using virtual warehouse: "PYDATA_LAB_WH"


### Create External Stages Using Python API

Stages define locations where **data files** are stored. We'll create an external stage that points to a public S3 bucket. This bucket will contain the CSV data files containing the raw data we want to use to build our pipeline.

Using cloud object storage (e.g., AWS S3 bucket) for storing raw data files is a very common practice in data engineering. A common pattern is to land raw data files in cloud object storage and use data platforms to ingest the data in the data files into the data platform.

![Stages](./img/stages.png)

In [ ]:
# Create external stage
tasty_bytes_stage = Stage(
    name="tasty_bytes_stage",
    url="s3://sfquickstarts/tasty-bytes-builder-education/",
    directory_table=StageDirectoryTable(enable=True)
)

root.databases[database_name].schemas["raw"].stages.create(
    tasty_bytes_stage,
    mode=CreateMode.or_replace
)
print("External stage 'tasty_bytes_stage' created successfully")

External stage 'tasty_bytes_stage' created successfully


### Create Raw Tables Using Python API

Let's now define the tables that will hold our raw data. We'll define three rwa tables, and together, you can think of them as the "raw zone" within our data platform, containing the data we loaded from the source CSV files in AWS S3.

The three "raw zone" tables:

1. `order_header` - Order-level information

2. `order_detail` - Line item information

3. `menu` - Product catalog

Across these three tables, there are nearly **1 billion rows of raw data.**

In [ ]:
# Get schema reference
raw_schema_ref = db.schemas["raw"]

# Define order_header table structure
order_header_columns = [
    TableColumn(name="order_id", datatype="NUMBER(38,0)"),
    TableColumn(name="truck_id", datatype="NUMBER(38,0)"),
    TableColumn(name="location_id", datatype="FLOAT"),
    TableColumn(name="customer_id", datatype="NUMBER(38,0)"),
    TableColumn(name="discount_id", datatype="VARCHAR(16777216)"),
    TableColumn(name="shift_id", datatype="NUMBER(38,0)"),
    TableColumn(name="shift_start_time", datatype="TIME(9)"),
    TableColumn(name="shift_end_time", datatype="TIME(9)"),
    TableColumn(name="order_channel", datatype="VARCHAR(16777216)"),
    TableColumn(name="order_ts", datatype="TIMESTAMP_NTZ(9)"),
    TableColumn(name="served_ts", datatype="VARCHAR(16777216)"),
    TableColumn(name="order_currency", datatype="VARCHAR(3)"),
    TableColumn(name="order_amount", datatype="NUMBER(38,4)"),
    TableColumn(name="order_tax_amount", datatype="VARCHAR(16777216)"),
    TableColumn(name="order_discount_amount", datatype="VARCHAR(16777216)"),
    TableColumn(name="order_total", datatype="NUMBER(38,4)")
]

# Create order_header table using Python API
order_header_table = Table(
    name="order_header",
    columns=order_header_columns
)

raw_schema_ref.tables.create(order_header_table, mode=CreateMode.or_replace)
print("Table 'order_header' created successfully")

Table 'order_header' created successfully


In [ ]:
# Define order_detail table structure
order_detail_columns = [
    TableColumn(name="order_detail_id", datatype="NUMBER(38,0)"),
    TableColumn(name="order_id", datatype="NUMBER(38,0)"),
    TableColumn(name="menu_item_id", datatype="NUMBER(38,0)"),
    TableColumn(name="discount_id", datatype="VARCHAR(16777216)"),
    TableColumn(name="line_number", datatype="NUMBER(38,0)"),
    TableColumn(name="quantity", datatype="NUMBER(5,0)"),
    TableColumn(name="unit_price", datatype="NUMBER(38,4)"),
    TableColumn(name="price", datatype="NUMBER(38,4)"),
    TableColumn(name="order_item_discount_amount", datatype="VARCHAR(16777216)")
]

# Create order_detail table
order_detail_table = Table(
    name="order_detail",
    columns=order_detail_columns
)

raw_schema_ref.tables.create(order_detail_table, mode=CreateMode.or_replace)
print("Table 'order_detail' created successfully")

Table 'order_detail' created successfully


In [ ]:
# Define menu table structure
menu_columns = [
    TableColumn(name="menu_id", datatype="NUMBER(19,0)"),
    TableColumn(name="menu_type_id", datatype="NUMBER(38,0)"),
    TableColumn(name="menu_type", datatype="VARCHAR(16777216)"),
    TableColumn(name="truck_brand_name", datatype="VARCHAR(16777216)"),
    TableColumn(name="menu_item_id", datatype="NUMBER(38,0)"),
    TableColumn(name="menu_item_name", datatype="VARCHAR(16777216)"),
    TableColumn(name="item_category", datatype="VARCHAR(16777216)"),
    TableColumn(name="item_subcategory", datatype="VARCHAR(16777216)"),
    TableColumn(name="cost_of_goods_usd", datatype="NUMBER(38,4)"),
    TableColumn(name="sale_price_usd", datatype="NUMBER(38,4)"),
    TableColumn(name="menu_item_health_metrics_obj", datatype="VARIANT")
]

# Create menu table
menu_table = Table(
    name="menu",
    columns=menu_columns
)

raw_schema_ref.tables.create(menu_table, mode=CreateMode.or_replace)
print("Table 'menu' created successfully")

Table 'menu' created successfully


## **Part 2: Ingest data into data objects**

### Populate the raw zone tables with data using COPY_INTO_TABLE

Now we'll load data from the files in external stage into our raw tables using Snowpark's `copy_into_table` method.

The dataset contains approximately 1 billion records across all tables. This may take a few minutes.

In [ ]:
# Load order_header data using Snowpark copy_into_table
print("Loading order_header data...")

order_header_df = session.read.csv("@tasty_bytes_db.raw.tasty_bytes_stage/raw_pos/order_header/")

# Map $1, $2, $3... to the actual columns by position
result = order_header_df.copy_into_table(
    "tasty_bytes_db.raw.order_header",  
    target_columns=["order_id", "truck_id", "location_id", "customer_id", "discount_id", 
                    "shift_id", "shift_start_time", "shift_end_time", "order_channel", 
                    "order_ts", "served_ts", "order_currency", "order_amount", 
                    "order_tax_amount", "order_discount_amount", "order_total"],
    transformations=["$1", "$2", "$3", "$4", "$5", "$6", "$7", "$8", 
                     "$9", "$10", "$11", "$12", "$13", "$14", "$15", "$16"]
)

print(f"Loaded {len(result)} batch(es) into order_header")

Loading order_header data...
Loaded 179 batch(es) into order_header


In [ ]:
# Load order_detail data using Snowpark copy_into_table
print("Loading order_detail data...")

order_detail_df = session.read.csv("@tasty_bytes_db.raw.tasty_bytes_stage/raw_pos/order_detail/")

result = order_detail_df.copy_into_table(
    "tasty_bytes_db.raw.order_detail",
    target_columns=["order_detail_id", "order_id", "menu_item_id", "discount_id", 
                    "line_number", "quantity", "unit_price", "price", "order_item_discount_amount"],
    transformations=["$1", "$2", "$3", "$4", "$5", "$6", "$7", "$8", "$9"]
)

print(f"Successfully loaded {len(result)} batch(es) into order_detail")

Loading order_detail data...
Successfully loaded 270 batch(es) into order_detail


In [ ]:
# Load menu data using Snowpark copy_into_table
print("Loading menu data...")

menu_df = session.read.csv("@tasty_bytes_db.raw.tasty_bytes_stage/raw_pos/menu/")

result = menu_df.copy_into_table(
    "tasty_bytes_db.raw.menu",
    target_columns=["menu_id", "menu_type_id", "menu_type", "truck_brand_name", "menu_item_id",
                    "menu_item_name", "item_category", "item_subcategory", "cost_of_goods_usd",
                    "sale_price_usd", "menu_item_health_metrics_obj"],
    transformations=["$1", "$2", "$3", "$4", "$5", "$6", "$7", "$8", "$9", "$10", "$11"]
)

print(f"Successfully loaded {len(result)} batch(es) into menu")

Loading menu data...
Successfully loaded 1 batch(es) into menu


### Verify Data Loading Using Snowpark

We've already verified the data was loaded correctly by checking row counts in the Snowflake UI. But we can do this programmatically as well using Snowpark.

In [ ]:
# Check row counts using Snowpark
order_header_count = session.table("tasty_bytes_db.raw.order_header").count()
order_detail_count = session.table("tasty_bytes_db.raw.order_detail").count()
menu_count = session.table("tasty_bytes_db.raw.menu").count()

print(f"order_header table: {order_header_count:,} rows")
print(f"order_detail table: {order_detail_count:,} rows")
print(f"menu table: {menu_count:,} rows")

order_header table: 248,201,269 rows
order_detail table: 673,655,465 rows
menu table: 100 rows


In [ ]:
# View sample data from order_header using Snowpark (you can do this with the other table names too)
print("Sample data from order_header:")
session.table("tasty_bytes_db.raw.order_header").limit(5).show()

Sample data from order_header:
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ORDER_ID"  |"TRUCK_ID"  |"LOCATION_ID"  |"CUSTOMER_ID"  |"DISCOUNT_ID"  |"SHIFT_ID"  |"SHIFT_START_TIME"  |"SHIFT_END_TIME"  |"ORDER_CHANNEL"  |"ORDER_TS"           |"SERVED_TS"  |"ORDER_CURRENCY"  |"ORDER_AMOUNT"  |"ORDER_TAX_AMOUNT"  |"ORDER_DISCOUNT_AMOUNT"  |"ORDER_TOTAL"  |
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|60938729    |191         |11541.0        |NULL           |NULL           |189043      |15:00:00            |22:00:00      

### Review data pipeline architecture

Let's quickly review the reference architecture diagram to understand what we've built.

## **Part 3: Transform raw data into insights**

### What are Dynamic Tables?

Dynamic Tables are Snowflake's declarative approach to data transformation. Instead of writing imperative code (mutiple step-by-step scripts, tasks, streams, stored procedures) to manage data pipelines, you define the desired end state once and Snowflake handles:

- Scheduling and orchestration: how often to refresh your transformed data
- Incremental refresh logic: process only changed rows in source tables during refreshes
- Automatic dependency tracking: native DAG in the Snowflake UI (we will see this)

### Create Tier 1 Dynamic Table - Orders Enriched

Now we begin building our transformations in our pipeline. The first tier dynamic tables enriches raw data with derived attributes.

The `orders_enriched` dynamic table:
- Extracts temporal dimensions (date, day of week, hour)
- Converts discount amounts to numbers
- Adds a discount flag

We set `target_lag` to 12 hours (43200 seconds) as an example. This setting means Snowflake will refresh this table to be at most 12 hours behind the source data (the tables in our raw zone).

#### Build the dynamic table in Python:

In [34]:
# Build orders_enriched Dynamic Table query using Snowpark DataFrame API
orders_enriched_df = session.table("tasty_bytes_db.raw.order_header") \
    .select(
        # Order identifiers
        F.col("order_id"),
        F.col("truck_id"),
        F.col("customer_id"),
        F.col("order_channel"),
        # Temporal dimensions
        F.col("order_ts").alias("order_timestamp"),
        F.to_date(F.col("order_ts")).alias("order_date"),
        F.call_builtin("DAYNAME", F.col("order_ts")).alias("day_name"),
        F.hour(F.col("order_ts")).alias("order_hour"),
        # Financial metrics
        F.col("order_amount"),
        F.col("order_total"),
        F.call_builtin("TRY_TO_NUMBER", F.col("order_discount_amount"), F.lit(10), F.lit(2)).alias("order_discount_amount"),
        # Simple discount flag
        F.when(
            (F.col("discount_id").isNotNull()) & (F.col("discount_id") != ""),
            F.lit(True)
        ).otherwise(F.lit(False)).alias("has_discount")
    ) \
    .where(
        F.col("order_id").isNotNull() & F.col("order_ts").isNotNull()
    )

# Create the dynamic table
orders_enriched_df.create_or_replace_dynamic_table(
    name="tasty_bytes_db.analytics.orders_enriched",
    warehouse="PYDATA_LAB_WH",
    lag="12 hours"
)

print("Tier 1 dynamic table 'orders_enriched' created successfully")


Tier 1 dynamic table 'orders_enriched' created successfully


#### ...or build the dynamic table in SQL:

In [ ]:
# Create orders_enriched dynamic table query
orders_enriched_query = """
SELECT
    -- Order identifiers
    order_id,
    truck_id,
    customer_id,
    order_channel,
    -- Temporal dimensions
    order_ts AS order_timestamp,
    DATE(order_ts) AS order_date,
    DAYNAME(order_ts) AS day_name,
    HOUR(order_ts) AS order_hour,
    -- Financial metrics
    order_amount,
    order_total,
    TRY_TO_NUMBER(order_discount_amount, 10, 2) AS order_discount_amount,
    -- Simple discount flag
    CASE
        WHEN discount_id IS NOT NULL AND discount_id != '' THEN TRUE
        ELSE FALSE
    END AS has_discount
FROM tasty_bytes_db.raw.order_header
WHERE order_id IS NOT NULL
    AND order_ts IS NOT NULL
"""

orders_enriched_dt = DynamicTable(
    name="orders_enriched_sql",
    target_lag=UserDefinedLag(seconds=43200),  # 12 hours
    warehouse="PYDATA_LAB_WH",
    query=orders_enriched_query
)

root.databases[database_name].schemas["analytics"].dynamic_tables.create(
    orders_enriched_dt,
    mode=CreateMode.or_replace
)

print("Tier 1 dynamic table 'orders_enriched_sql' created successfully")

### Create Tier 1 Dynamic Table - Order Items Enriched

The `order_items_enriched` dynamic table:

- Joins order details with the menu to get product information
- Calculates profit metrics (unit profit, line profit, profit margin)
- Adds discount information

This table also uses a 12-hour target lag.

#### Build the dynamic table in Python:

In [36]:
# Build order_items_enriched using Snowpark DataFrame API
od = session.table("tasty_bytes_db.raw.order_detail")
m = session.table("tasty_bytes_db.raw.menu")

order_items_enriched_df = od.join(m, od["menu_item_id"] == m["menu_item_id"], "inner") \
    .select(
        # Order detail identifiers
        od["order_detail_id"],
        od["order_id"],
        od["line_number"],
        # Product information
        od["menu_item_id"].alias("menu_item_id"),
        m["menu_item_name"],
        m["item_category"],
        m["item_subcategory"],
        m["truck_brand_name"],
        m["menu_type"],
        # Quantity and pricing
        od["quantity"],
        od["unit_price"],
        od["price"].alias("line_total"),
        m["cost_of_goods_usd"],
        m["sale_price_usd"],
        # Profit calculations
        (od["unit_price"] - m["cost_of_goods_usd"]).alias("unit_profit"),
        ((od["unit_price"] - m["cost_of_goods_usd"]) * od["quantity"]).alias("line_profit"),
        F.when(
            od["unit_price"] > 0,
            F.round(((od["unit_price"] - m["cost_of_goods_usd"]) / od["unit_price"]) * 100, 2)
        ).otherwise(0).alias("profit_margin_pct"),
        # Discount information
        F.call_builtin("TRY_TO_NUMBER", od["order_item_discount_amount"], F.lit(10), F.lit(2)).alias("line_discount_amount"),
        F.when(
            (od["discount_id"].isNotNull()) & (od["discount_id"] != ""),
            F.lit(True)
        ).otherwise(F.lit(False)).alias("has_discount")
    ) \
    .where(
        od["order_id"].isNotNull() & od["menu_item_id"].isNotNull()
    )

# Create the dynamic table
order_items_enriched_df.create_or_replace_dynamic_table(
    name="tasty_bytes_db.analytics.order_items_enriched",
    warehouse="PYDATA_LAB_WH",
    lag="12 hours"
)

print("Tier 1 dynamic table 'order_items_enriched' created successfully")

Tier 1 dynamic table 'order_items_enriched' created successfully


#### ...or build the dynamic table in SQL:

In [ ]:
# Create order_items_enriched dynamic table
order_items_enriched_query = """
SELECT
    -- Order detail identifiers
    od.order_detail_id,
    od.order_id,
    od.line_number,
    -- Product information
    od.menu_item_id,
    m.menu_item_name,
    m.item_category,
    m.item_subcategory,
    m.truck_brand_name,
    m.menu_type,
    -- Quantity and pricing
    od.quantity,
    od.unit_price,
    od.price AS line_total,
    m.cost_of_goods_usd,
    m.sale_price_usd,
    -- Profit calculations
    (od.unit_price - m.cost_of_goods_usd) AS unit_profit,
    (od.unit_price - m.cost_of_goods_usd) * od.quantity AS line_profit,
    CASE
        WHEN od.unit_price > 0 THEN
            ROUND(((od.unit_price - m.cost_of_goods_usd) / od.unit_price) * 100, 2)
        ELSE 0
    END AS profit_margin_pct,
    -- Discount information
    TRY_TO_NUMBER(od.order_item_discount_amount, 10, 2) AS line_discount_amount,
    CASE
        WHEN od.discount_id IS NOT NULL AND od.discount_id != '' THEN TRUE
        ELSE FALSE
    END AS has_discount
FROM tasty_bytes_db.raw.order_detail od
INNER JOIN tasty_bytes_db.raw.menu m
    ON od.menu_item_id = m.menu_item_id
WHERE od.order_id IS NOT NULL
    AND od.menu_item_id IS NOT NULL
"""

order_items_enriched_dt = DynamicTable(
    name="order_items_enriched_sql",
    target_lag=UserDefinedLag(seconds=43200),  # 12 hours
    warehouse="PYDATA_LAB_WH",
    query=order_items_enriched_query
)

root.databases[database_name].schemas["analytics"].dynamic_tables.create(
    order_items_enriched_dt,
    mode=CreateMode.or_replace
)

print("Tier 1 dynamic table 'order_items_enriched_sql' created successfully")

### Verify Tier 1 Dynamic Tables Using Snowpark

Let's verify that our Tier 1 dynamic tables were created and populated correctly using Snowpark DataFrames.

In [37]:
# Check orders_enriched using Snowpark
print("Sample data from orders_enriched:")

# Replace with orders_enriched_sql if needed
session.table("tasty_bytes_db.analytics.orders_enriched") \
    .select("order_id", "order_date", "day_name", "order_hour", "order_amount", "order_total", "has_discount") \
    .limit(10).show()

Sample data from orders_enriched:
-----------------------------------------------------------------------------------------------------------
|"ORDER_ID"  |"ORDER_DATE"  |"DAY_NAME"  |"ORDER_HOUR"  |"ORDER_AMOUNT"  |"ORDER_TOTAL"  |"HAS_DISCOUNT"  |
-----------------------------------------------------------------------------------------------------------
|398127561   |2020-09-17    |Thu         |13            |13.0000         |13.0000        |False           |
|398127688   |2020-09-17    |Thu         |17            |22.0000         |22.0000        |False           |
|398127815   |2020-09-17    |Thu         |18            |13.0000         |13.0000        |False           |
|398127942   |2020-09-17    |Thu         |19            |26.0000         |26.0000        |False           |
|398128069   |2020-09-17    |Thu         |20            |13.0000         |13.0000        |False           |
|398128196   |2020-09-18    |Fri         |8             |87.0000         |87.0000        |False       

In [38]:
# Check order_items_enriched using Snowpark
print("Sample data from order_items_enriched:")

# Replace with orders_items_enriched_sql if needed
session.table("tasty_bytes_db.analytics.order_items_enriched") \
    .select("menu_item_name", "item_category", "quantity", "unit_price", "line_total", "line_profit", "profit_margin_pct") \
    .limit(10).show()

Sample data from order_items_enriched:
------------------------------------------------------------------------------------------------------------------------------
|"MENU_ITEM_NAME"         |"ITEM_CATEGORY"  |"QUANTITY"  |"UNIT_PRICE"  |"LINE_TOTAL"  |"LINE_PROFIT"  |"PROFIT_MARGIN_PCT"  |
------------------------------------------------------------------------------------------------------------------------------
|The Salad of All Salads  |Main             |1           |12.0000       |12.0000       |6.0000         |50.00                |
|Bottled Soda             |Beverage         |1           |3.0000        |3.0000        |2.5000         |83.33                |
|Seitan Buffalo Wings     |Snack            |1           |7.0000        |7.0000        |3.0000         |42.86                |
|The Salad of All Salads  |Main             |1           |12.0000       |12.0000       |6.0000         |50.00                |
|Veggie Burger            |Main             |1           |9.0000        

### Create Tier 2 Dynamic Table - Order Fact

Tier 2 joins the enriched order headers with enriched order items to create a comprehensive fact table.

Note the `target_lag=DownstreamLag()` setting. This tells Snowflake to refresh this table whenever its upstream dependencies (Tier 1 tables) are refreshed. This creates an automatic dependency graph.

#### Build the dynamic table in Python:

In [39]:
# Build order_fact query using Snowpark DataFrame API
o = session.table("tasty_bytes_db.analytics.orders_enriched")
oi = session.table("tasty_bytes_db.analytics.order_items_enriched")

order_fact_df = o.join(oi, o["order_id"] == oi["order_id"], "inner") \
    .select(
        # Order header fields
        o["order_id"].alias("order_id"),
        o["truck_id"],
        o["customer_id"],
        o["order_channel"],
        o["order_timestamp"],
        o["order_date"],
        o["day_name"],
        o["order_hour"],
        o["order_amount"],
        o["order_total"],
        o["order_discount_amount"].alias("order_level_discount"),
        o["has_discount"].alias("order_has_discount"),
        # Order line item fields
        oi["order_detail_id"],
        oi["line_number"],
        oi["menu_item_id"],
        oi["menu_item_name"],
        oi["item_category"],
        oi["item_subcategory"],
        oi["truck_brand_name"],
        oi["menu_type"],
        oi["quantity"],
        oi["unit_price"],
        oi["line_total"],
        oi["cost_of_goods_usd"],
        oi["sale_price_usd"],
        oi["unit_profit"],
        oi["line_profit"],
        oi["profit_margin_pct"],
        oi["line_discount_amount"],
        oi["has_discount"].alias("line_has_discount")
    )

# Create the dynamic table
order_fact_df.create_or_replace_dynamic_table(
    name="tasty_bytes_db.analytics.order_fact",
    warehouse="PYDATA_LAB_WH",
    lag="DOWNSTREAM"
)

print("Tier 2 Dynamic table 'order_fact' created successfully")

Tier 2 Dynamic table 'order_fact' created successfully


#### ...or build the dynamic table in SQL:

In [ ]:
# Create order_fact dynamic table query
order_fact_query = """
SELECT
    -- Order header fields
    o.order_id,
    o.truck_id,
    o.customer_id,
    o.order_channel,
    o.order_timestamp,
    o.order_date,
    o.day_name,
    o.order_hour,
    o.order_amount,
    o.order_total,
    o.order_discount_amount AS order_level_discount,
    o.has_discount AS order_has_discount,
    -- Order line item fields
    oi.order_detail_id,
    oi.line_number,
    oi.menu_item_id,
    oi.menu_item_name,
    oi.item_category,
    oi.item_subcategory,
    oi.truck_brand_name,
    oi.menu_type,
    oi.quantity,
    oi.unit_price,
    oi.line_total,
    oi.cost_of_goods_usd,
    oi.sale_price_usd,
    oi.unit_profit,
    oi.line_profit,
    oi.profit_margin_pct,
    oi.line_discount_amount,
    oi.has_discount AS line_has_discount
FROM tasty_bytes_db.analytics.orders_enriched_sql o
INNER JOIN tasty_bytes_db.analytics.order_items_enriched_sql oi
    ON o.order_id = oi.order_id
"""

order_fact_dt = DynamicTable(
    name="order_fact_sql",
    target_lag=DownstreamLag(),  # Refresh when upstream tables refresh
    warehouse="PYDATA_LAB_WH",
    query=order_fact_query
)

root.databases[database_name].schemas["analytics"].dynamic_tables.create(
    order_fact_dt,
    mode=CreateMode.or_replace
)

print("Tier 2 Dynamic table 'order_fact_sql' created successfully")

### Verify Tier 2 Table Using Snowpark

Let's check the order_fact table to ensure it contains the joined data.

In [40]:
# Check order_fact using Snowpark
print("Sample data from order_fact:")

# Replace with order_fact_sql if needed
session.table("tasty_bytes_db.analytics.order_fact") \
    .select("order_id", "order_date", "menu_item_name", "item_category", "quantity", "order_total", "line_profit", "profit_margin_pct") \
    .limit(10).show()

Sample data from order_fact:
-------------------------------------------------------------------------------------------------------------------------------------
|"ORDER_ID"  |"ORDER_DATE"  |"MENU_ITEM_NAME"  |"ITEM_CATEGORY"  |"QUANTITY"  |"ORDER_TOTAL"  |"LINE_PROFIT"  |"PROFIT_MARGIN_PCT"  |
-------------------------------------------------------------------------------------------------------------------------------------
|63879385    |2022-10-26    |Combo Fried Rice  |Main             |3           |60.0000        |18.0000        |54.55                |
|63879385    |2022-10-26    |Wonton Soup       |Main             |4           |60.0000        |16.0000        |66.67                |
|63879385    |2022-10-26    |Bottled Soda      |Beverage         |1           |60.0000        |2.5000         |83.33                |
|63879622    |2022-10-26    |Combo Lo Mein     |Main             |1           |13.0000        |7.0000         |53.85                |
|63879696    |2022-10-26    |Comb

### Create Tier 3 Dynamic Table - Daily Business Metrics

Tier 3 contains aggregated metrics. The first is `daily_business_metrics`, which pre-aggregates key business KPIs by day.

This table aggregates:
- Volume metrics (orders, trucks, customers, items sold)
- Revenue metrics (total revenue, average order value)
- Profit metrics (total profit, average margin)
- Discount metrics (orders with discounts, discount amounts)

It also uses `TARGET_LAG = 'DOWNSTREAM'` to refresh when order_fact changes.

In [41]:
# Build daily_business_metrics using Snowpark DataFrame API
daily_business_metrics_df = session.table("tasty_bytes_db.analytics.order_fact") \
    .group_by("order_date", "day_name") \
    .agg(
        # Volume metrics
        F.count_distinct(F.col("order_id")).alias("total_orders"),
        F.count_distinct(F.col("truck_id")).alias("active_trucks"),
        F.count_distinct(F.col("customer_id")).alias("unique_customers"),
        F.sum(F.col("quantity")).alias("total_items_sold"),
        # Revenue metrics
        F.sum(F.col("order_total")).alias("total_revenue"),
        F.round(F.avg(F.col("order_total")), 2).alias("avg_order_value"),
        F.sum(F.col("line_total")).alias("total_line_item_revenue"),
        # Profit metrics
        F.sum(F.col("line_profit")).alias("total_profit"),
        F.round(F.avg(F.col("profit_margin_pct")), 2).alias("avg_profit_margin_pct"),
        # Discount metrics
        F.sum(F.when(F.col("order_has_discount"), 1).otherwise(0)).alias("orders_with_discount"),
        F.sum(F.col("order_level_discount")).alias("total_order_discount_amount"),
        F.sum(F.col("line_discount_amount")).alias("total_line_discount_amount")
    )

# Create the dynamic table using Snowpark DataFrame method
daily_business_metrics_df.create_or_replace_dynamic_table(
    name="tasty_bytes_db.analytics.daily_business_metrics",
    warehouse="PYDATA_LAB_WH",
    lag="DOWNSTREAM"
)

print("(Tier 3) Dynamic table 'daily_business_metrics' created successfully")

(Tier 3) Dynamic table 'daily_business_metrics' created successfully


#### ...or build the dynamic table in SQL:

In [ ]:
# Create daily_business_metrics dynamic table
daily_business_metrics_query = """
SELECT
    order_date,
    day_name,
    -- Volume metrics
    COUNT(DISTINCT order_id) AS total_orders,
    COUNT(DISTINCT truck_id) AS active_trucks,
    COUNT(DISTINCT customer_id) AS unique_customers,
    SUM(quantity) AS total_items_sold,
    -- Revenue metrics
    SUM(order_total) AS total_revenue,
    ROUND(AVG(order_total), 2) AS avg_order_value,
    SUM(line_total) AS total_line_item_revenue,
    -- Profit metrics
    SUM(line_profit) AS total_profit,
    ROUND(AVG(profit_margin_pct), 2) AS avg_profit_margin_pct,
    -- Discount metrics
    SUM(CASE WHEN order_has_discount THEN 1 ELSE 0 END) AS orders_with_discount,
    SUM(order_level_discount) AS total_order_discount_amount,
    SUM(line_discount_amount) AS total_line_discount_amount
FROM tasty_bytes_db.analytics.order_fact_sql
GROUP BY order_date, day_name
"""

daily_business_metrics_dt = DynamicTable(
    name="daily_business_metrics_sql",
    target_lag=DownstreamLag(),  # Refresh when upstream tables refresh
    warehouse="PYDATA_LAB_WH",
    query=daily_business_metrics_query
)

root.databases[database_name].schemas["analytics"].dynamic_tables.create(
    daily_business_metrics_dt,
    mode=CreateMode.or_replace
)

print("(Tier 3) Dynamic table 'daily_business_metrics_sql' created successfully")

### Create Tier 3 Dynamic Table - Product Performance Metrics

The second Tier 3 table is `product_performance_metrics`, which aggregates sales and profit data by product.

This table provides:
- Product dimensions (name, category, subcategory, brand)
- Sales volume (order count, units sold)
- Revenue and profit totals
- Performance indicators (revenue per unit, profit per unit)

In [42]:
# Build product_performance_metrics using Snowpark DataFrame API
product_performance_metrics_df = session.table("tasty_bytes_db.analytics.order_fact") \
    .group_by(
        "menu_item_id",
        "menu_item_name",
        "item_category",
        "item_subcategory",
        "truck_brand_name",
        "menu_type"
    ) \
    .agg(
        # Sales volume metrics
        F.count_distinct(F.col("order_id")).alias("order_count"),
        F.sum(F.col("quantity")).alias("total_units_sold"),
        # Revenue and profit metrics
        F.sum(F.col("line_total")).alias("total_revenue"),
        F.sum(F.col("line_profit")).alias("total_profit"),
        F.round(F.avg(F.col("unit_price")), 2).alias("avg_unit_price"),
        F.round(F.avg(F.col("profit_margin_pct")), 2).alias("avg_profit_margin_pct"),
        # Cost metrics
        F.avg(F.col("cost_of_goods_usd")).alias("avg_cogs"),
        F.avg(F.col("sale_price_usd")).alias("standard_sale_price"),
        # Performance indicators
        (F.sum(F.col("line_total")) / F.call_builtin("NULLIF", F.sum(F.col("quantity")), F.lit(0))).alias("revenue_per_unit"),
        (F.sum(F.col("line_profit")) / F.call_builtin("NULLIF", F.sum(F.col("quantity")), F.lit(0))).alias("profit_per_unit")
    )

# Create the dynamic table using Snowpark DataFrame method
product_performance_metrics_df.create_or_replace_dynamic_table(
    name="tasty_bytes_db.analytics.product_performance_metrics",
    warehouse="PYDATA_LAB_WH",
    lag="DOWNSTREAM"
)

print("(Tier 3) Dynamic table 'product_performance_metrics created successfully")

(Tier 3) Dynamic table 'product_performance_metrics created successfully


#### ...or build the dynamic table in SQL:

In [ ]:
# Create product_performance_metrics dynamic table
product_performance_metrics_query = """
SELECT
    -- Product dimensions
    menu_item_id,
    menu_item_name,
    item_category,
    item_subcategory,
    truck_brand_name,
    menu_type,
    -- Sales volume metrics
    COUNT(DISTINCT order_id) AS order_count,
    SUM(quantity) AS total_units_sold,
    -- Revenue and profit metrics
    SUM(line_total) AS total_revenue,
    SUM(line_profit) AS total_profit,
    ROUND(AVG(unit_price), 2) AS avg_unit_price,
    ROUND(AVG(profit_margin_pct), 2) AS avg_profit_margin_pct,
    -- Cost metrics
    AVG(cost_of_goods_usd) AS avg_cogs,
    AVG(sale_price_usd) AS standard_sale_price,
    -- Performance indicators
    SUM(line_total) / NULLIF(SUM(quantity), 0) AS revenue_per_unit,
    SUM(line_profit) / NULLIF(SUM(quantity), 0) AS profit_per_unit
FROM tasty_bytes_db.analytics.order_fact_sql
GROUP BY
    menu_item_id,
    menu_item_name,
    item_category,
    item_subcategory,
    truck_brand_name,
    menu_type
"""

product_performance_metrics_dt = DynamicTable(
    name="product_performance_metrics_sql",
    target_lag=DownstreamLag(),  # Refresh when upstream tables refresh
    warehouse="PYDATA_LAB_WH",
    query=product_performance_metrics_query
)

root.databases[database_name].schemas["analytics"].dynamic_tables.create(
    product_performance_metrics_dt,
    mode=CreateMode.or_replace
)

print("(Tier 3) Dynamic table 'product_performance_metrics_sql' created successfully")

### Verify Tier 3 Tables Using Snowpark

Let's examine the aggregated metrics to ensure they're calculating correctly using Snowpark DataFrames.

In [43]:
# Check daily_business_metrics using Snowpark
print("Sample data from daily_business_metrics:")

# Replace with daily_business_metrics_sql if needed
session.table("tasty_bytes_db.analytics.daily_business_metrics") \
    .select("order_date", "day_name", "total_orders", "unique_customers", "total_revenue", "avg_order_value", "total_profit", "avg_profit_margin_pct") \
    .order_by(F.col("order_date").desc()) \
    .limit(10).show()

Sample data from daily_business_metrics:
----------------------------------------------------------------------------------------------------------------------------------------------------
|"ORDER_DATE"  |"DAY_NAME"  |"TOTAL_ORDERS"  |"UNIQUE_CUSTOMERS"  |"TOTAL_REVENUE"  |"AVG_ORDER_VALUE"  |"TOTAL_PROFIT"  |"AVG_PROFIT_MARGIN_PCT"  |
----------------------------------------------------------------------------------------------------------------------------------------------------
|2022-11-01    |Tue         |425886          |18644               |59793523.2500    |51.75              |8961352.6000    |58.66                    |
|2022-10-31    |Mon         |412664          |15854               |58254392.2500    |51.90              |8705875.3000    |58.67                    |
|2022-10-30    |Sun         |459105          |18020               |64367132.7500    |51.62              |9647234.4500    |58.68                    |
|2022-10-29    |Sat         |465857          |18145              

In [44]:
# Check product_performance_metrics using Snowpark
print("Sample data from product_performance_metrics (top products by revenue):")

# Replace with product_performance_metrics_sql if needed
session.table("tasty_bytes_db.analytics.product_performance_metrics") \
    .select("menu_item_name", "item_category", "order_count", "total_units_sold", "total_revenue", "total_profit", "avg_profit_margin_pct") \
    .order_by(F.col("total_revenue").desc()) \
    .limit(10).show()

Sample data from product_performance_metrics (top products by revenue):
--------------------------------------------------------------------------------------------------------------------------------------------------
|"MENU_ITEM_NAME"            |"ITEM_CATEGORY"  |"ORDER_COUNT"  |"TOTAL_UNITS_SOLD"  |"TOTAL_REVENUE"  |"TOTAL_PROFIT"  |"AVG_PROFIT_MARGIN_PCT"  |
--------------------------------------------------------------------------------------------------------------------------------------------------
|The King Combo              |Main             |12627298       |21563360            |431267200.0000   |172506880.0000  |40.00                    |
|Tandoori Mixed Grill        |Main             |12097473       |20659415            |371869470.0000   |144615905.0000  |38.89                    |
|Lean Chicken Tikka Masala   |Main             |12098077       |20660126            |351222142.0000   |144620882.0000  |41.18                    |
|Spicy Miso Vegetable Ramen  |Main            

### List Dynamic Tables

In [45]:
# List all dynamic tables in the analytics schema
analytics_schema_ref = root.databases["tasty_bytes_db"].schemas["analytics"]
dynamic_table_collection = DynamicTableCollection(analytics_schema_ref)
dynamic_tables = dynamic_table_collection.iter()
print("Dynamic tables in 'analytics' schema:")
for dt in dynamic_tables:
    print(dt.name)

Dynamic tables in 'analytics' schema:
DAILY_BUSINESS_METRICS
ORDERS_ENRICHED
ORDER_FACT
ORDER_ITEMS_ENRICHED
PRODUCT_PERFORMANCE_METRICS


## Part 4: Incremental refresh for dynamic tables 

To demonstrate incremental refresh, we need a way to add new data to our raw tables. We'll create a stored procedure that generates synthetic orders.

This procedure:
- Takes a parameter for the number of orders to generate
- Creates synthetic order headers with randomized values
- Creates corresponding order details (line items)
- Maintains referential integrity between orders and line items

Note: Stored procedure creation currently requires SQL.

In [ ]:
# Create stored procedure to generate demo orders
session.sql("""
    CREATE OR REPLACE PROCEDURE tasty_bytes_db.raw.generate_demo_orders(num_rows INTEGER)
    RETURNS STRING
    LANGUAGE SQL
    AS
    $$
    DECLARE
        orders_before INTEGER;
        orders_after INTEGER;
        orders_inserted INTEGER;
        details_before INTEGER;
        details_after INTEGER;
        details_inserted INTEGER;
    BEGIN
        -- Capture counts before insert
        SELECT COUNT(*) INTO :orders_before FROM tasty_bytes_db.raw.order_header;
        SELECT COUNT(*) INTO :details_before FROM tasty_bytes_db.raw.order_detail;
        
        -- Create temporary table with new order IDs to maintain referential integrity
        CREATE OR REPLACE TEMPORARY TABLE new_orders AS
        SELECT
            (1000000 + UNIFORM(1, 999999, RANDOM()))::NUMBER(38,0) AS new_order_id,
            oh.order_id AS original_order_id,
            oh.truck_id,
            oh.location_id,
            oh.customer_id,
            oh.discount_id,
            oh.shift_id,
            oh.shift_start_time,
            oh.shift_end_time,
            oh.order_channel,
            DATEADD('day', DATEDIFF('day', oh.order_ts, CURRENT_DATE()), oh.order_ts) AS order_ts,
            oh.served_ts,
            oh.order_currency,
            oh.order_amount * (0.8 + UNIFORM(0, 0.4, RANDOM())) AS order_amount,
            oh.order_tax_amount,
            oh.order_discount_amount,
            oh.order_total * (0.8 + UNIFORM(0, 0.4, RANDOM())) AS order_total
        FROM tasty_bytes_db.raw.order_header oh
        WHERE oh.order_id IS NOT NULL
        ORDER BY RANDOM()
        LIMIT :num_rows;
        
        -- Insert synthetic order headers
        INSERT INTO tasty_bytes_db.raw.order_header (
            order_id, truck_id, location_id, customer_id, discount_id, shift_id,
            shift_start_time, shift_end_time, order_channel, order_ts, served_ts,
            order_currency, order_amount, order_tax_amount, order_discount_amount,
            order_total
        )
        SELECT
            new_order_id, truck_id, location_id, customer_id, discount_id, shift_id,
            shift_start_time, shift_end_time, order_channel, order_ts, served_ts,
            order_currency, order_amount, order_tax_amount, order_discount_amount,
            order_total
        FROM new_orders;
        
        -- Insert corresponding order details (line items)
        INSERT INTO tasty_bytes_db.raw.order_detail (
            order_detail_id, order_id, menu_item_id, discount_id, line_number,
            quantity, unit_price, price, order_item_discount_amount
        )
        SELECT
            (2000000 + UNIFORM(1, 9999999, RANDOM()))::NUMBER(38,0) AS order_detail_id,
            no.new_order_id AS order_id,
            od.menu_item_id,
            od.discount_id,
            od.line_number,
            od.quantity,
            od.unit_price * (0.8 + UNIFORM(0, 0.4, RANDOM())) AS unit_price,
            od.price * (0.8 + UNIFORM(0, 0.4, RANDOM())) AS price,
            od.order_item_discount_amount
        FROM new_orders no
        INNER JOIN tasty_bytes_db.raw.order_detail od
            ON no.original_order_id = od.order_id;
        
        -- Capture counts after insert
        SELECT COUNT(*) INTO :orders_after FROM tasty_bytes_db.raw.order_header;
        SELECT COUNT(*) INTO :details_after FROM tasty_bytes_db.raw.order_detail;
        
        orders_inserted := :orders_after - :orders_before;
        details_inserted := :details_after - :details_before;
        
        -- Clean up temporary table
        DROP TABLE IF EXISTS new_orders;
        
        RETURN 'Successfully generated ' || orders_inserted::STRING || ' new orders with ' ||
               details_inserted::STRING || ' line items. Total orders: ' || orders_after::STRING;
    END;
    $$
""").collect()

print("Stored procedure 'generate_demo_orders' created")

### Capture Initial State Using Snowpark

Before we add new data, let's capture the current state of our raw tables using Snowpark.

In [46]:
# Get current row counts using Snowpark
initial_order_count = session.table("tasty_bytes_db.raw.order_header").count()
initial_detail_count = session.table("tasty_bytes_db.raw.order_detail").count()

print(f"Initial order_header count: {initial_order_count:,}")
print(f"Initial order_detail count: {initial_detail_count:,}")

# Get latest order timestamp using Snowpark
latest_order = session.table("tasty_bytes_db.raw.order_header") \
    .select(F.max("order_ts").alias("latest_order_ts")) \
    .collect()[0][0]
print(f"Latest order timestamp: {latest_order}")

Initial order_header count: 248,201,269
Initial order_detail count: 673,655,465
Latest order timestamp: 2022-11-01 22:59:59


### Generate New Demo Orders Using Stored Procedure

Now we'll call the stored procedure to generate 1200 new orders. This simulates new data arriving in our system.

We use Snowpark's `session.call()` method to invoke the stored procedure.

In [47]:
# Call stored procedure to generate 1200 new orders
result = session.call("tasty_bytes_db.raw.generate_demo_orders", 1200)
print(result)

SnowparkSQLException: (1304): 01c0ea61-0207-38f6-0000-001979733429: 002141 (42601): SQL compilation error:
Unknown user-defined function TASTY_BYTES_DB.RAW.GENERATE_DEMO_ORDERS.

### Verify New Data Using Snowpark

Let's confirm the new data was inserted into our raw tables using Snowpark DataFrames.

In [ ]:
# Get updated row counts
new_order_count = session.table("tasty_bytes_db.raw.order_header").count()
new_detail_count = session.table("tasty_bytes_db.raw.order_detail").count()

print(f"New order_header count: {new_order_count:,} (+{new_order_count - initial_order_count:,} rows)")
print(f"New order_detail count: {new_detail_count:,} (+{new_detail_count - initial_detail_count:,} rows)")

### Manually Trigger Incremental Refresh - Tier 1

Dynamic tables refresh automatically based on their TARGET_LAG, but we can manually trigger a refresh to see immediate results.

We'll refresh the Tier 1 tables first, which will detect and process only the new data (incremental refresh) in the raw zone tables.

Note: ALTER DYNAMIC TABLE commands currently require SQL.

In [ ]:
# Refresh Tier 1 tables
# Append _sql if you want to refresh those created with SQL queries

print("Refreshing orders_enriched...")
session.sql("ALTER DYNAMIC TABLE tasty_bytes_db.analytics.orders_enriched REFRESH").collect()

print("Refreshing order_items_enriched...")
session.sql("ALTER DYNAMIC TABLE tasty_bytes_db.analytics.order_items_enriched REFRESH").collect()

print("Tier 1 tables refreshed")

### Check Tier 1 Refresh History

We can query the refresh history to see whether Snowflake performed an INCREMENTAL or FULL refresh.

Incremental refresh processes only changed data, which is much faster and more efficient than a full refresh.

In [ ]:
# Check refresh history for orders_enriched
# Append _sql to table name if you want to check those created with SQL queries

print("Refresh history for orders_enriched:")
session.sql("""
    SELECT name, refresh_action, state, refresh_start_time, refresh_trigger
    FROM TABLE(INFORMATION_SCHEMA.DYNAMIC_TABLE_REFRESH_HISTORY(
        NAME => 'tasty_bytes_db.ANALYTICS.ORDERS_ENRICHED'
    ))
    ORDER BY refresh_start_time DESC 
    LIMIT 5
""").show()

In [ ]:
# Check refresh history for order_items_enriched
# Append _sql to table name if you want to check those created with SQL queries

print("Refresh history for order_items_enriched:")
session.sql("""
    SELECT name, refresh_action, state, refresh_start_time, refresh_trigger
    FROM TABLE(INFORMATION_SCHEMA.DYNAMIC_TABLE_REFRESH_HISTORY(
        NAME => 'tasty_bytes_db.ANALYTICS.ORDER_ITEMS_ENRICHED'
    ))
    ORDER BY refresh_start_time DESC 
    LIMIT 5
""").show()

### Manually Trigger Refresh - Tier 2 and Tier 3 Tables

Now we'll refresh the downstream tables (Tier 2 and Tier 3). Because they use `TARGET_LAG = 'DOWNSTREAM'`, they will automatically refresh when their upstream dependencies change.

In [ ]:
# Refresh Tier 2 table
# Append _sql if you want to refresh those created with SQL queries

print("Refreshing order_fact...")
session.sql("ALTER DYNAMIC TABLE tasty_bytes_db.analytics.order_fact REFRESH").collect()

# Refresh Tier 3 tables
print("Refreshing daily_business_metrics...")
session.sql("ALTER DYNAMIC TABLE tasty_bytes_db.analytics.daily_business_metrics REFRESH").collect()

print("Refreshing product_performance_metrics...")
session.sql("ALTER DYNAMIC TABLE tasty_bytes_db.analytics.product_performance_metrics REFRESH").collect()

print("All downstream tables refreshed")

### Check Refresh History for Tier 2 and Tier 3 tables

Let's examine the refresh history for Tier 2 and Tier 3 tables to see the incremental refresh in action.

In [ ]:
# Check refresh history for order_fact
# Append _sql to table name if you want to check those created with SQL queries

print("Refresh history for order_fact:")
session.sql("""
    SELECT name, refresh_action, state, refresh_start_time
    FROM TABLE(INFORMATION_SCHEMA.DYNAMIC_TABLE_REFRESH_HISTORY(
        NAME => 'tasty_bytes_db.ANALYTICS.ORDER_FACT'
    ))
    ORDER BY refresh_start_time DESC 
    LIMIT 5
""").show()

In [ ]:
# Check refresh history for daily_business_metrics
# Append _sql to table name if you want to check those created with SQL queries

print("Refresh history for daily_business_metrics:")
session.sql("""
    SELECT name, refresh_action, state, refresh_start_time
    FROM TABLE(INFORMATION_SCHEMA.DYNAMIC_TABLE_REFRESH_HISTORY(
        NAME => 'tasty_bytes_db.ANALYTICS.DAILY_BUSINESS_METRICS'
    ))
    ORDER BY refresh_start_time DESC 
    LIMIT 5
""").show()

In [ ]:
# Check refresh history for product_performance_metrics
# Append _sql to table name if you want to check those created with SQL queries

print("Refresh history for product_performance_metrics:")
session.sql("""
    SELECT name, refresh_action, state, refresh_start_time
    FROM TABLE(INFORMATION_SCHEMA.DYNAMIC_TABLE_REFRESH_HISTORY(
        NAME => 'tasty_bytes_db.ANALYTICS.PRODUCT_PERFORMANCE_METRICS'
    ))
    ORDER BY refresh_start_time DESC 
    LIMIT 5
""").show()

### Verify Updated Metrics Using Snowpark

Let's query our aggregated metrics using Snowpark to confirm that the new data has propagated through the entire pipeline.

In [ ]:
# Check updated daily metrics using Snowpark
# Append _sql to table name if you want to check those created with SQL queries

print("Updated daily business metrics (most recent dates):")
session.table("tasty_bytes_db.analytics.daily_business_metrics") \
    .select("order_date", "total_orders", "total_items_sold", "total_revenue", "total_profit") \
    .order_by(F.col("order_date").desc()) \
    .limit(5).show()

In [ ]:
# Check updated product performance using Snowpark
# Append _sql to table name if you want to check those created with SQL queries

print("Updated product performance (top products by revenue):")
session.table("tasty_bytes_db.analytics.product_performance_metrics") \
    .select("menu_item_name", "item_category", "total_units_sold", "total_revenue", "total_profit") \
    .order_by(F.col("total_revenue").desc()) \
    .limit(10).show()

### Summarize Latest Refresh Operations

Let's create a summary view of the most recent refresh for each dynamic table, showing:
- Refresh type (INCREMENTAL vs FULL)
- State (SUCCEEDED, FAILED, etc.)
- Duration in seconds

In [ ]:
# Summary of latest refresh operations
print("Latest refresh operations for all dynamic tables:")
session.sql("""
    SELECT
        name,
        refresh_action,
        state,
        refresh_start_time,
        refresh_end_time,
        DATEDIFF('second', refresh_start_time, refresh_end_time) AS refresh_duration_seconds
    FROM (
        SELECT 
            name, 
            refresh_action, 
            state, 
            refresh_start_time, 
            refresh_end_time,
            ROW_NUMBER() OVER (PARTITION BY name ORDER BY refresh_start_time DESC) as rn
        FROM TABLE(INFORMATION_SCHEMA.DYNAMIC_TABLE_REFRESH_HISTORY())
    )
    WHERE rn = 1
    ORDER BY name
""").show()

## Part 5: Create an AI agent to extract insights from data in dynamic tables


### Create infrastructure for agent

We'll use Snowflake Intelligence to create and use AI agents that can answer natural language questions about your data. First, we'll create the necessary infrastructure.

In [ ]:
# Grant required privileges (requires ACCOUNTADMIN)
session.use_role("ACCOUNTADMIN")
session.sql("GRANT CREATE DATABASE ON ACCOUNT TO ROLE pydata_lab_role").collect()

# Switch back to pydata_lab_role
session.use_role("pydata_lab_role")
print("Privileges granted for Snowflake Intelligence")

# Create Intelligence database using Python API
intelligence_db = Database(name="snowflake_intelligence")
root.databases.create(intelligence_db, mode=CreateMode.or_replace)
print("Database 'snowflake_intelligence' created")

# Create agents schema using Python API
intelligence_db_ref = root.databases["snowflake_intelligence"]
agents_schema = Schema(name="agents")
intelligence_db_ref.schemas.create(agents_schema, mode=CreateMode.or_replace)
print("Schema 'agents' created")

# Grant permissions
session.sql("GRANT USAGE ON DATABASE snowflake_intelligence TO ROLE pydata_lab_role").collect()
session.sql("GRANT USAGE ON SCHEMA snowflake_intelligence.agents TO ROLE pydata_lab_role").collect()
session.sql("GRANT CREATE AGENT ON SCHEMA snowflake_intelligence.agents TO ROLE pydata_lab_role").collect()

print("Agent infrastructure created")

### Walk through the UI with the instructor to create an agent

### Sample Questions for Intelligence Agent

Once you create an agent through the UI and connect it to your semantic model, you can ask questions like:

1. What was the total revenue for the last 30 days?
2. Which products have the highest profit margins?
3. Show me daily revenue trends as a line chart
4. How many unique customers did we have yesterday?
5. What percentage of orders include discounts?
6. Compare revenue by product category
7. Show me the top 5 products by profit
8. What are the busiest hours for orders?
9. How does revenue vary by day of week?
10. Which truck brands generate the most profit?

### Verify the agent responses with the following queries

In [ ]:
# Get top 10 products by revenue using Snowpark
print("Top 10 products by revenue (using Snowpark):")
top_products = session.table("tasty_bytes_db.analytics.product_performance_metrics") \
    .select(
        F.col("menu_item_name"),
        F.col("item_category"),
        F.col("total_revenue"),
        F.col("total_profit"),
        F.col("avg_profit_margin_pct")
    ) \
    .order_by(F.col("total_revenue").desc()) \
    .limit(10)

top_products.show()

In [ ]:
# Calculate revenue by category using Snowpark aggregation
print("Revenue by product category (using Snowpark):")
category_revenue = session.table("tasty_bytes_db.analytics.product_performance_metrics") \
    .group_by("item_category") \
    .agg(
        F.sum("total_revenue").alias("category_revenue"),
        F.sum("total_profit").alias("category_profit"),
        F.sum("total_units_sold").alias("units_sold")
    ) \
    .order_by(F.col("category_revenue").desc())

category_revenue.show()

In [ ]:
# Get daily revenue trend for last 30 days using Snowpark
print("Daily revenue trend (last 30 days):")
daily_trend = session.table("tasty_bytes_db.analytics.daily_business_metrics") \
    .select(
        F.col("order_date"),
        F.col("total_orders"),
        F.col("total_revenue"),
        F.col("total_profit"),
        F.col("unique_customers")
    ) \
    .order_by(F.col("order_date").desc()) \
    .limit(30)

daily_trend.show()

## Cleanup (Optional)

When you're done with this lab, you can clean up all resources.

WARNING: This will delete all data and objects created in this notebook. Only run this cell if you're sure you want to remove everything.

In [ ]:
# Uncomment the following lines to perform cleanup

# # Drop databases
# root.databases["tasty_bytes_db"].delete()
# print("Database 'tasty_bytes_db' dropped")

# root.databases["snowflake_intelligence"].delete()
# print("Database 'snowflake_intelligence' dropped")

# # Drop warehouse
# root.warehouses["PYDATA_LAB_WH"].delete()
# print("Warehouse 'PYDATA_LAB_WH' dropped")

# # Optionally drop the role
# # session.use_role("ACCOUNTADMIN")
# # session.sql("DROP ROLE IF EXISTS pydata_lab_role").collect()
# # print("Role 'pydata_lab_role' dropped")

print("Cleanup section ready (uncomment to execute)")

In [ ]:
# Close the session
session.close()
print("Session closed.")

## Review and Conclusion

Let's recap what we've built:

1. **End-to-end data pipeline** that:

- Ingests raw data from CSV files in an S3 bucket and lands it as data in tables
- Transforms data automatically with declarative dynamic tables that process only incremental changes
- Delivers a semantic model that can be used for an AI agent to extract insights from data

2. **No Orchestration Code**: No need for tasks, custom scheduling logic, or third-party scheduling tools

### What you used to build the pipeline

- Cloud object storage (AWS S3) to hold raw data files

- **Snowflake Python APIs** (`snowflake.core`) for creating databases, schemas, warehouses, and tables, and for ingesting data into Snowflake

- **Snowpark DataFrames** for querying data

- (Some) **SQL** only when necessary (dynamic tables, stored procedures, stages) where Snowflake Python API support is limited

- Semantic models (YAML files) to configure an AI agent for data insights


### Additional Resources

- Snowflake Developer Hub: [developers.snowflake.com](developers.snowflake.com)

- **Snowflake-Labs**, open source GitHub organization: [https://github.com/Snowflake-Labs](https://github.com/Snowflake-Labs)

- [Snowflake Developers YouTube Channel](https://www.youtube.com/channel/UCxgY7r-o_ql8ADIdyiQr3Zw)

- [**Coursera: Snowflake Data Engineering Professional Certificate:**](https://www.coursera.org/professional-certificates/snowflake-data-engineering)

- [Snowflake Dynamic Tables Documentation](https://docs.snowflake.com/en/user-guide/dynamic-tables-intro)

- [Snowpark Python Developer Guide](https://docs.snowflake.com/en/developer-guide/snowpark/python/index)

- [Snowflake Python API Reference](https://docs.snowflake.com/developer-guide/snowflake-python-api/reference/latest/index)

- [Snowflake Intelligence](https://docs.snowflake.com/en/user-guide/snowflake-cortex/snowflake-intelligence)

### Connect with me on LinkedIn

[https://www.linkedin.com/in/gilberto-hernandez](https://www.linkedin.com/in/gilberto-hernandez)